# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64\Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
from cc_ddpg_agent import Agents
from collections import deque
import torch

n_episodes=1
max_t=1
desired_score=30

agents = Agents(num_agents, state_size, action_size, 4109)

scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores
score = 0

for i_episode in range(1, n_episodes+1): 

    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    score = np.zeros(num_agents)                          # initialize the score (for each agent)

    for t in range(max_t):
        actions     = agents.act(states)                       # select an action (for each agent)
        env_info    = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards     = env_info.rewards                         # get reward (for each agent)
        dones       = env_info.local_done                        # see if episode finished

        agents.step(states, actions, rewards, next_states, dones)
        
        states = next_states                               # roll over states to next time step
        score += env_info.rewards                         # update the score (for each agent)
        
        if np.any(dones):                                  # exit loop if episode finished
            break
     
    episode_score = np.mean(score)            # score averaged over agents    
    scores_window.append(episode_score)       # save most recent episode score
    scores.append(episode_score)              # save most recent episode score
    #print('Total score (averaged over agents) this episode: {}'.format())
    print('\rEpisode {}\tAverage Score (over agents): {:.2f}'.format(i_episode, episode_score), end="")
    if i_episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    if np.mean(scores_window)>=desired_score:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
        torch.save(agents.actor_local.state_dict(),   'checkpoint.pth')
        torch.save(agents.actor_target.state_dict(),  'backup_actor_target.pth')
        torch.save(agents.critic_local.state_dict(),  'backup_critic_local.pth')
        torch.save(agents.critic_target.state_dict(), 'backup_critic_target.pth')
        break


self.actor_local(states) tensor(1.00000e-02 *
       [[-9.1621,  7.3101, -1.0981, -3.7512],
        [-9.1592,  7.2982, -1.0979, -3.8348],
        [-8.6296,  6.9680, -0.4974, -4.1725],
        [-8.8840,  7.0379, -0.7281, -3.7234],
        [-9.1362,  7.2854, -1.1237, -3.7574],
        [-8.6349,  6.9465, -0.4770, -4.2003],
        [-8.7237,  7.0367, -0.9192, -4.2569],
        [-8.9526,  6.9858, -0.6863, -3.7135],
        [-9.1217,  7.1012, -0.7903, -3.6222],
        [-8.6403,  7.1142, -0.7072, -4.2691],
        [-8.6304,  6.9574, -0.4969, -4.1780],
        [-8.6689,  7.0843, -0.8425, -4.2838],
        [-9.0802,  7.0318, -0.7940, -3.5873],
        [-9.1077,  7.2580, -1.0741, -3.9142],
        [-9.1289,  7.1411, -0.8398, -3.6346],
        [-8.6245,  7.0969, -0.5995, -4.2688],
        [-9.1106,  7.0625, -0.7667, -3.6211],
        [-8.8676,  7.0608, -0.7441, -3.7279],
        [-8.9078,  7.1217, -0.9484, -4.0991],
        [-9.1567,  7.2962, -1.0818, -3.7123]])
self.actor_local(states).cpu() te

When finished, you can close the environment.

In [ ]:
env.close()

### 4. Was training successful?
If training was successful, the desired score has been achieved.
Let's see how: episode by episode (on average).  
If training failed, run it again (you could also play with parameters before).

In [ ]:
#import numpy as np
!pip3 install box2d

import matplotlib.pyplot as plt
%matplotlib inline

!python -m pip install pyvirtualdisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

# TESTING
### 5. Agent in action!
Let's see how the trained agent play!
We'll load the trained model, checkpoint.pth .

Note: this step can be run apart from the previous TRAINING steps; checkpoint.pth contains a pre-trained model.


In [1]:
from unityagents import UnityEnvironment
import numpy as np

# version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='Reacher_Windows_x86_64\Reacher.exe')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [2]:
from cc_ddpg_agent import Agents
import torch 

agents = Agents(num_agents, state_size, action_size, 4109)
agents.load_to_cpu(path='checkpoint.pth')

In [3]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
score = np.zeros(num_agents)                          # initialize the score (for each agent)

max_t = 1000
for t in range(max_t):
    actions     = agents.act(states)                       # select an action (for each agent)
    env_info    = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards     = env_info.rewards                         # get reward (for each agent)
    dones       = env_info.local_done                        # see if episode finished
    states = next_states                               # roll over states to next time step
    score += env_info.rewards                         # update the score (for each agent)

    if np.any(dones):                                  # exit loop if episode finished
        break

episode_score = np.mean(score)            # score averaged over agents    
print('\rEpisode  Score (over agents): {:.2f}'.format(episode_score), end="")


Episode  Score (over agents): 36.72

In [4]:
env.close()